<a href="https://colab.research.google.com/github/MiraGlaceon/lessons-and-pets/blob/main/HW_Lesson15_Churn_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание: Логистическая регрессия для классификации оттока клиентов

Цель задания: самостоятельно пройти все этапы решения задачи бинарной классификации с помощью логистической регрессии.

Датасет: Bank Customer Churn (отток клиентов банка).

Задача: предсказать, уйдет ли клиент из банка (закроет счет) или останется.

Что нужно сделать:
1. Загрузить и изучить данные.
2. Провести предобработку (очистка, кодирование категориальных признаков).
3. Разделить данные на обучающую и тестовую выборки.
4. Обучить модель логистической регрессии.
5. Оценить качество модели по метрикам accuracy, precision, recall, F1-score.
6. Интерпретировать результаты и сделать выводы.


# Шаг 1. Импорт библиотек

Задание: импортируйте все необходимые библиотеки для работы с данными, обучения модели и оценки качества.

Что нужно импортировать:
- pandas для работы с таблицами
- numpy для числовых операций
- train_test_split для разделения данных
- LogisticRegression для модели
- accuracy_score, precision_score, recall_score, f1_score, confusion_matrix для метрик


In [24]:
# Импортируйте необходимые библиотеки здесь
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

from sklearn.preprocessing import StandardScaler


# Шаг 2. Загрузка данных

Датасет: Bank Customer Churn

Ссылка на данные:
https://raw.githubusercontent.com/YBI-Foundation/Dataset/main/Bank%20Churn%20Modelling.csv

Задание:
1. Загрузите датасет с помощью pd.read_csv.
2. Выведите первые 5 строк с помощью метода head().
3. Проверьте размер датасета с помощью len() или shape.

Описание столбцов:
- Attrition_Flag: целевая переменная (Existing Customer остался, Attrited Customer ушел)
- Customer_Age: возраст клиента
- Gender: пол клиента
- Dependent_count: количество иждивенцев
- Education_Level: уровень образования
- Marital_Status: семейное положение
- Income_Category: категория дохода
- Card_Category: тип карты
- Months_on_book: сколько месяцев клиент с банком
- Total_Relationship_Count: количество продуктов банка, которыми пользуется клиент
- Months_Inactive_12_mon: количество неактивных месяцев за последний год
- Contacts_Count_12_mon: сколько раз клиент обращался в банк за последний год
- Credit_Limit: кредитный лимит
- Total_Revolving_Bal: общий возобновляемый баланс
- Avg_Open_To_Buy: средняя доступная сумма для покупок
- Total_Amt_Chng_Q4_Q1: изменение суммы транзакций между 4 и 1 кварталом
- Total_Trans_Amt: общая сумма транзакций за последние 12 месяцев
- Total_Trans_Ct: общее количество транзакций за последние 12 месяцев
- Total_Ct_Chng_Q4_Q1: изменение количества транзакций между 4 и 1 кварталом
- Avg_Utilization_Ratio: средний коэффициент использования кредитной линии


In [8]:
# Загрузите данные и выведите первые строки
url = "https://raw.githubusercontent.com/YBI-Foundation/Dataset/main/Bank%20Churn%20Modelling.csv"
df = pd.read_csv(url)

print('Первые 5 строк')
print(df.head())
print('Размер датасета (строки, столбцы)')
print(df.shape)
print('Количество строк:', len(df))




Первые 5 строк
   CustomerId   Surname  CreditScore Geography  Gender  Age  Tenure  \
0    15634602  Hargrave          619    France  Female   42       2   
1    15647311      Hill          608     Spain  Female   41       1   
2    15619304      Onio          502    France  Female   42       8   
3    15701354      Boni          699    France  Female   39       1   
4    15737888  Mitchell          850     Spain  Female   43       2   

     Balance  Num Of Products  Has Credit Card  Is Active Member  \
0       0.00                1                1                 1   
1   83807.86                1                0                 1   
2  159660.80                3                1                 0   
3       0.00                2                0                 0   
4  125510.82                1                1                 1   

   Estimated Salary  Churn  
0         101348.88      1  
1         112542.58      0  
2         113931.57      1  
3          93826.63      0  
4   

# Шаг 3. Первичный анализ данных

Задание:
1. Выполните метод info() для просмотра типов данных и пропусков.
2. Выполните describe(include='all') для базовой статистики.
3. Проверьте распределение целевой переменной Attrition_Flag с помощью value_counts().

Вопросы для анализа:
- Сколько всего клиентов в датасете? - 10000
- Есть ли пропущенные значения? - Нет
- Какая доля клиентов ушла из банка? - 0,2037 (20%)
- Сбалансированы ли классы? - Нет, класс 0 - 80%


In [9]:
# Выполните первичный анализ данных
df.info()
df.describe(include='all')
df['Churn'].value_counts()
df['Churn'].value_counts(normalize=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerId        10000 non-null  int64  
 1   Surname           10000 non-null  object 
 2   CreditScore       10000 non-null  int64  
 3   Geography         10000 non-null  object 
 4   Gender            10000 non-null  object 
 5   Age               10000 non-null  int64  
 6   Tenure            10000 non-null  int64  
 7   Balance           10000 non-null  float64
 8   Num Of Products   10000 non-null  int64  
 9   Has Credit Card   10000 non-null  int64  
 10  Is Active Member  10000 non-null  int64  
 11  Estimated Salary  10000 non-null  float64
 12  Churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


,proportion
Churn,
0,0.7963
1,0.2037


# Шаг 4. Предобработка данных

Задание:
1. Удалите ненужные столбцы:
   - CLIENTNUM (идентификатор клиента, не несет информации для модели)
   - Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
   - Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
   (это технические столбцы, они не нужны)

2. Преобразуйте целевую переменную Attrition_Flag:
   - Existing Customer -> 0 (клиент остался)
   - Attrited Customer -> 1 (клиент ушел)

3. Определите, какие столбцы являются категориальными (тип object), а какие числовыми.

4. Примените one-hot encoding к категориальным признакам с помощью pd.get_dummies().
   Используйте параметр drop_first=True, чтобы избежать мультиколлинеарности.

Объяснение:
- One-hot encoding превращает категориальные признаки в набор бинарных столбцов.
- Например, столбец Gender с значениями M и F превратится в Gender_M (1 если мужчина, 0 если женщина).
- drop_first=True удаляет одну категорию из каждого признака, чтобы избежать избыточности.


In [17]:
# Удалите ненужные столбцы
df = df.drop(columns=['CustomerId', 'Surname'])


KeyError: "['CustomerId', 'Surname'] not found in axis"

In [18]:
# Преобразуйте целевую переменную
# В этом датасете уже закодирована целевая переменная:
# 0 — клиент остался
# 1 — клиент ушёл
y = df['Churn']


In [19]:
# Определите категориальные признаки
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(exclude=['object']).columns

categorical_features, numerical_features


(Index(['Geography', 'Gender'], dtype='object'),
 Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'Num Of Products',
        'Has Credit Card', 'Is Active Member', 'Estimated Salary', 'Churn'],
       dtype='object'))

In [20]:
# Примените one-hot encoding
X = df.drop(columns='Churn')

X = pd.get_dummies(
    X,
    columns=categorical_features,
    drop_first=True
)
X.head()


,CreditScore,Age,Tenure,Balance,Num Of Products,Has Credit Card,Is Active Member,Estimated Salary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,False,True,False


# Шаг 5. Подготовка данных для обучения

Задание:
1. Создайте матрицу признаков X:
   - Удалите из датасета столбец с целевой переменной Attrition_Flag.
   - Преобразуйте результат в numpy массив с помощью .values.

2. Создайте вектор целевой переменной y:
   - Выделите столбец Attrition_Flag.
   - Преобразуйте в numpy массив.

3. Выведите размеры X и y с помощью .shape.

Объяснение:
- X содержит все признаки (независимые переменные), по которым модель будет делать предсказания.
- y содержит целевую переменную (зависимую переменную), которую модель будет предсказывать.
- Размер X должен быть (количество клиентов, количество признаков).
- Размер y должен быть (количество клиентов,).


In [21]:
# Создайте X и y
X = X.values
y = y.values

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (10000, 11)
y shape: (10000,)


# Шаг 6. Разделение на обучающую и тестовую выборки

Задание:
1. Разделите данные на train и test с помощью train_test_split.
2. Используйте следующие параметры:
   - test_size=0.2 (20% данных для теста, 80% для обучения)
   - random_state=42 (для воспроизводимости результатов)
   - stratify=y (чтобы пропорции классов в train и test были одинаковыми)

3. Выведите размеры полученных массивов.

Объяснение:
- train данные используются для обучения модели.
- test данные используются для оценки качества модели на новых данных.
- stratify=y гарантирует, что в обеих выборках будет примерно одинаковое соотношение ушедших и оставшихся клиентов.
- random_state=42 делает разделение повторяемым (если запустить код снова, получим те же самые train и test).


In [22]:
# Разделите данные на train и test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (8000, 11)
X_test shape: (2000, 11)
y_train shape: (8000,)
y_test shape: (2000,)


# Шаг 7. Обучение модели логистической регрессии

Задание:
1. Создайте объект модели LogisticRegression с параметром max_iter=1000.
2. Обучите модель на обучающих данных с помощью метода fit(X_train, y_train).
3. Выведите сообщение об успешном обучении.

Объяснение:
- LogisticRegression это алгоритм для бинарной классификации.
- max_iter=1000 задает максимальное число итераций для сходимости алгоритма оптимизации.
- Метод fit обучает модель на данных X_train (признаки) и y_train (целевая переменная).
- После обучения модель готова делать предсказания.


In [27]:
# Создайте и обучите модель
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

# Шаг 8. Предсказания на тестовой выборке

Задание:
1. Сделайте предсказания для тестовой выборки с помощью метода predict(X_test).
2. Получите вероятности предсказаний с помощью метода predict_proba(X_test).
3. Выведите первые 10 предсказаний и первые 10 реальных значений для сравнения.

Объяснение:
- predict(X_test) возвращает предсказанный класс для каждого клиента (0 или 1).
- predict_proba(X_test) возвращает вероятности для каждого класса. Первый столбец вероятность класса 0, второй вероятность класса 1.
- Сравнение предсказаний с реальными значениями помогает визуально оценить, насколько хорошо работает модель.


In [28]:
# Сделайте предсказания
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)

print("Предсказанные значения (первые 10):", y_pred[:10])
print("Реальные значения (первые 10):", y_test[:10])

print("\nВероятности (первые 10):")
y_pred_proba[:10]



Предсказанные значения (первые 10): [0 1 0 0 0 0 0 1 0 0]
Реальные значения (первые 10): [0 0 0 0 0 0 0 0 0 0]

Вероятности (первые 10):


array([[0.90889862, 0.09110138],
       [0.49366368, 0.50633632],
       [0.8979418 , 0.1020582 ],
       [0.82638458, 0.17361542],
       [0.88651992, 0.11348008],
       [0.79919938, 0.20080062],
       [0.88849735, 0.11150265],
       [0.21619813, 0.78380187],
       [0.83178373, 0.16821627],
       [0.74689392, 0.25310608]])

5. Оценить качество модели по метрикам accuracy, precision, recall, F1-score.

In [29]:
# Accuracy
acc = accuracy_score(y_test, y_pred)

# Precision (точность для класса 1 — ушедшие клиенты)
prec = precision_score(y_test, y_pred)

# Recall (полнота для класса 1)
rec = recall_score(y_test, y_pred)

# F1-score
f1 = f1_score(y_test, y_pred)

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(acc, 3))
print("Precision:", round(prec, 3))
print("Recall:", round(rec, 3))
print("F1-score:", round(f1, 3))
print("\nConfusion Matrix:\n", cm)

Accuracy: 0.782
Precision: 0.433
Recall: 0.221
F1-score: 0.293

Confusion Matrix:
 [[1475  118]
 [ 317   90]]


6. Интерпретировать результаты и сделать выводы.

Изначально данные были не хорошо классифицированны, 80% - клиенты банка, 20% - ушли. Это могло повлиять на результаты предсказания, потому что 0 выпадал значительно чаще. К примеру, на 10ти предсказаниях получилось 2 неверных. По метрикам модель получилась достаточно точной с accuracy 0.782 (хотя на несбалансированных данных, это могут быть не верные метрики). Precision: 0.433 также показывает, что модель сделала предсказания, которые реально сбылись на 43%. По итогу модель хорошо определяет оставшихся клиентов и плохо определяет тех, кто ушел